In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

def connectSQL(server_name,database):
    # Replace 'your_database' and 'your_server' with your actual SQL Server and database name
    database_connection = create_engine(f'mssql+pyodbc://@{server_name}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
    print(f'Connecting to {server_name}, {database}')
    return database_connection

def uploadFileToSQL(database_connection,df,table_name):
    # Create a new table in the database
    df.to_sql(table_name, con=database_connection, if_exists='replace', index=False)
    print(f"Table '{table_name}' created and data imported successfully.")


In [2]:
### CREATE CONNECTION TO THE SERVER ###
server_name = 'your-server-name'
database = 'your-database-name'
database_connection = connectSQL(server_name,database)    


Connecting to C161LX084150214, MBVCS


In [4]:
### CREATE DATAFRAME TO UPLOAD TO THE SERVER ###

## 1. Part Category    
# Replace 'path_to_excel_file' with the path to your Excel file
excel_file_path = r'path-to-your-excel-file'
# Replace 'your_table_name' with the desired table name
table_name = 'Part Categories'
sheet_name = 'Total'
df = pd.read_excel(excel_file_path,sheet_name=sheet_name)
current_column_name = 'MC (1)'
new_column_name = 'MKTCD'
# Rename the column
df.rename(columns={current_column_name: new_column_name}, inplace=True)
df=df[['SPS_Bezeichnung','MKTCD']]
df = df.drop_duplicates(subset=['MKTCD'])
df = df.dropna(subset=['MKTCD'])
uploadFileToSQL(database_connection,df,table_name)


Table 'Part Categories' created and data imported successfully.


In [5]:
#2. Date table

# Replace 'path_to_excel_file' with the path to your Excel file
excel_file_path = r'path_to_excel_file'
# Replace 'your_table_name' with the desired table name
table_name = 'Date table'
sheet_name = 'Date table'
df = pd.read_excel(excel_file_path,sheet_name=sheet_name)
uploadFileToSQL(database_connection,df,table_name)

#3. Dealer table

# Replace 'path_to_excel_file' with the path to your Excel file
excel_file_path = r'path_to_excel_file'
# Replace 'your_table_name' with the desired table name
table_name = 'Dealer table'
sheet_name = 'Dealer table'
df = pd.read_excel(excel_file_path,sheet_name=sheet_name)
uploadFileToSQL(database_connection,df,table_name)

#4. Performance Summary

# Replace 'path_to_excel_file' with the path to your Excel file
excel_file_path = r'path_to_excel_file'
# Replace 'your_table_name' with the desired table name
table_name = 'Performance summary'
sheet_name = 'Performance summary'
df = pd.read_excel(excel_file_path,sheet_name=sheet_name)
uploadFileToSQL(database_connection,df,table_name)




Table 'Date table' created and data imported successfully.
Table 'Dealer table' created and data imported successfully.
Table 'Performance summary' created and data imported successfully.


In [7]:
# 5. Kerridge Data 
import os 

# Specify the folder path where subfolders with CSV files are located
folder_path = r'path_to_folder'

# Initialize an empty list to store DataFrames from each CSV file
dataframes = []

# Iterate through each subfolder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".csv"):
            # Create the full file path
            file_path = os.path.join(root, file)
            
            # Read the CSV file into a DataFrame and append it to the list
            df = pd.read_csv(file_path)
            dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
result_df = pd.concat(dataframes, ignore_index=True)
uploadFileToSQL(database_connection,result_df,table_name='YTD_kerridge_data')

Table 'YTD_kerridge_data' created and data imported successfully.
